In [9]:
%pip install sdv

  Using cached sdv-1.33.0-py3-none-any.whl.metadata (14 kB)
  Using cached boto3-1.42.39-py3-none-any.whl.metadata (6.8 kB)
  Using cached botocore-1.42.39-py3-none-any.whl.metadata (5.9 kB)
  Using cached cloudpickle-3.1.2-py3-none-any.whl.metadata (7.1 kB)
  Using cached graphviz-0.21-py3-none-any.whl.metadata (12 kB)
  Using cached pandas-2.3.3-cp313-cp313-win_amd64.whl.metadata (19 kB)
  Using cached copulas-0.14.0-py3-none-any.whl.metadata (9.7 kB)
  Using cached ctgan-0.12.0-py3-none-any.whl.metadata (10 kB)
  Using cached deepecho-0.8.0-py3-none-any.whl.metadata (11 kB)
  Using cached rdt-1.20.0-py3-none-any.whl.metadata (11 kB)
  Using cached sdmetrics-0.27.0-py3-none-any.whl.metadata (10 kB)
  Using cached pyyaml-6.0.3-cp313-cp313-win_amd64.whl.metadata (2.4 kB)
  Using cached jmespath-1.1.0-py3-none-any.whl.metadata (7.6 kB)
  Using cached s3transfer-0.16.0-py3-none-any.whl.metadata (1.7 kB)
  Using cached urllib3-2.6.3-py3-none-any.whl.metadata (6.9 kB)
  Using cached pytz-2

  You can safely remove it manually.


In [2]:
import pandas as pd

In [3]:
url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"

In [4]:
df = pd.read_csv(url)

In [5]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [7]:
# Check for missing values
print(f"Missing Ages: {df['Age'].isnull().sum()}")

# Check for fractional ages (numbers that aren't whole)
# This looks for ages where the decimal part is NOT zero
fractional_ages = df[df['Age'] % 1 != 0]
print(f"Fractional Ages (Babies): {len(fractional_ages)}")
print(fractional_ages['Age'].head())

Missing Ages: 177
Fractional Ages (Babies): 202
5    NaN
17   NaN
19   NaN
26   NaN
28   NaN
Name: Age, dtype: float64


In [8]:
print(f"Unique Pclass values: {df['Pclass'].nunique()}")
print(f"Unique Fare values:   {df['Fare'].nunique()}")

Unique Pclass values: 3
Unique Fare values:   248


In [13]:
from sdv.single_table import CTGANSynthesizer
from sdv.metadata import Metadata

print("✅ GAN engine Loaded Successfully")

✅ GAN engine Loaded Successfully


In [23]:
# 2. Autodetect the structure from dataframe
metadata = Metadata.detect_from_dataframe(data=df, table_name='titanic')

In [24]:
# Creating the Synthesizer
synthesizer = CTGANSynthesizer(
    metadata=metadata,
    epochs=100,
    verbose=True
)

# Training the AI
print("---TRAINING STARTED---")
synthesizer.fit(df)
print("---TRAINING COMPLETE")

d:\Akash\Codes\Projects\TensorVeil\.venv\Lib\site-packages\sdv\single_table\base.py:134: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(


---TRAINING STARTED---
PerformanceAlert: Using the CTGANSynthesizer on this data is not recommended. To model this data, CTGAN will generate a large number of columns.

Original Column Name   Est # of Columns (CTGAN)
Survived               2
Pclass                 3
Name                   891
Sex                    2
Age                    11
SibSp                  7
Parch                  7
Ticket                 681
Fare                   11
Cabin                  148
Embarked               4

We recommend preprocessing discrete columns that can have many values, using 'update_transformers'. Or you may drop columns that are not necessary to model. (Exit this script using ctrl-C)


Gen. (1.89) | Discrim. (0.03): 100%|██████████| 100/100 [00:39<00:00,  2.53it/s]

---TRAINING COMPLETE


In [26]:
# asking AI to generate fake columns
synthetic_data = synthesizer.sample(num_rows=100)
synthetic_data.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,13542083,1,3,"Hart, Mrs. Benjamin (Esther Ada Bloomfield)",male,27.43,1,0,315094,0.0000,NaN,C
1,14467049,1,1,"Palsson, Miss. Torborg Danira",female,NaN,0,0,370370,10.2819,NaN,C
2,16001953,0,3,"McGough, Mr. James Robert",female,0.42,0,0,350060,0.0000,NaN,S
3,9864820,1,3,"Marechal, Mr. Pierre",male,12.46,0,0,113501,47.7582,NaN,S
4,6645628,0,1,"Roebling, Mr. Washington Augustus II",male,28.92,0,1,347082,6.2168,NaN,C
5,15460863,1,3,"Ringhini, Mr. Sante",female,28.29,0,0,349209,41.7500,NaN,NaN
6,1929392,0,1,"Moran, Mr. James",female,12.94,0,0,C.A. 2673,13.2049,C47,S
7,4343376,1,3,"Simonius-Blumer, Col. Oberst Alfons",male,28.02,1,0,S.O./P.P. 3,3.1644,B50,C
8,14144949,1,2,"Lehmann, Miss. Bertha",male,0.42,3,0,19947,0.3726,C68,S
9,6301410,0,3,"Waelens, Mr. Achille",female,36.77,0,0,315093,41.1992,NaN,S


In [28]:
# Filling missing ages
mean_age = synthetic_data['Age'].mean()
synthetic_data['Age'] = synthetic_data['Age'].fillna(mean_age)

In [30]:
# Round the "Age" to the nearest whole number
synthetic_data['Age'] = synthetic_data['Age'].round().astype(int).clip(lower=1)

# Round the "Fare" to 2 decimal places
synthetic_data['Fare'] = synthetic_data['Fare'].round(2)

# Polished data
synthetic_data.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,13542083,1,3,"Hart, Mrs. Benjamin (Esther Ada Bloomfield)",male,27,1,0,315094,0.00,NaN,C
1,14467049,1,1,"Palsson, Miss. Torborg Danira",female,19,0,0,370370,10.28,NaN,C
2,16001953,0,3,"McGough, Mr. James Robert",female,1,0,0,350060,0.00,NaN,S
3,9864820,1,3,"Marechal, Mr. Pierre",male,12,0,0,113501,47.76,NaN,S
4,6645628,0,1,"Roebling, Mr. Washington Augustus II",male,29,0,1,347082,6.22,NaN,C
5,15460863,1,3,"Ringhini, Mr. Sante",female,28,0,0,349209,41.75,NaN,NaN
6,1929392,0,1,"Moran, Mr. James",female,13,0,0,C.A. 2673,13.20,C47,S
7,4343376,1,3,"Simonius-Blumer, Col. Oberst Alfons",male,28,1,0,S.O./P.P. 3,3.16,B50,C
8,14144949,1,2,"Lehmann, Miss. Bertha",male,1,3,0,19947,0.37,C68,S
9,6301410,0,3,"Waelens, Mr. Achille",female,37,0,0,315093,41.20,NaN,S
